In [42]:
from requests import Session
from requests.auth import HTTPBasicAuth
from zeep import Client
from zeep.wsse.username import UsernameToken
from zeep.transports import Transport
import time

# Credenciales de la API
username = 'yonatan.osorio@openenglish.com'
password = 'Colombia22++'
wsdl_url = f'https://api.five9.com/wsadmin/v13/AdminWebService?wsdl&user={username}'

try:
    # Crear sesión y cliente SOAP
    session = Session()
    session.auth = HTTPBasicAuth(username, password)
    transport = Transport(session=session)
    client = Client(wsdl=wsdl_url, transport=transport, wsse=UsernameToken(username, password))

    # Validar credenciales con una llamada real
    client.service.getApiVersions()
    print("✅ Conexión y autenticación exitosa con Five9.")

    # Paso 1: Ejecutar el reporte (runReport)
    folder_name = 'Agent Reports'
    report_name = 'Agents Information'

    criteria = {
        'time': {
            'start': '2025-04-30T00:00:00-05:00',
            'end': '2025-04-30T23:59:59-05:00'
        }
    }

    print("🚀 Ejecutando el reporte...")
    report_response = client.service.runReport(
        folderName=folder_name,
        reportName=report_name,
        criteria=criteria
    )

    report_id = report_response
    print(f"🆔 Reporte ejecutado. ID: {report_id}")

    # Paso 2: Esperar a que el reporte esté listo
    print("⏳ Esperando a que el reporte se complete...")
    report_ready = False
    while not report_ready:
        try:
            is_running = client.service.isReportRunning(identifier=report_id, timeout=60)
            if not is_running:
                report_ready = True
                print("✅ El reporte está listo.")
            else:
                print("⏳ El reporte aún está en ejecución. Esperando...")
                time.sleep(70)
        except Exception as e:
            print(f"❌ Error al verificar el estado del reporte: {e}")
            break

    # Paso 3: Descargar el reporte cuando esté listo
    if report_ready:
        print("📥 Descargando el reporte...")
        try:
            csv_result = client.service.getReportResultCsv(identifier=report_id)
            with open("reporte_five9.csv", "w", encoding="utf-8") as f:
                f.write(csv_result)
            print("✅ Reporte descargado y guardado como 'reporte_five9.csv'")
        except Exception as e:
            print(f"❌ Error al obtener el resultado del reporte: {e}")

except Exception as conn_error:
    print(f"❌ Error de conexión o autenticación con Five9: {conn_error}")


✅ Conexión y autenticación exitosa con Five9.
🚀 Ejecutando el reporte...
🆔 Reporte ejecutado. ID: 5FEB20097868BD630rt2.t.ie.oD7999@0p70629a3l6f4v095cEm
⏳ Esperando a que el reporte se complete...
✅ El reporte está listo.
📥 Descargando el reporte...
✅ Reporte descargado y guardado como 'reporte_five9.csv'
